In [57]:
import geopandas as gpd
import rioxarray as rxr
from rasterstats import zonal_stats
import pandas as pd
import os
import rasterio
from rasterio.mask import mask
import numpy as np
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from shapely.ops import unary_union
from rasterio.features import rasterize
from data_utils import best_res_align, align_r1_to_r2
import xarray as xr

In [58]:
laptop_or_server = 1
if laptop_or_server:
    # laptop folder
    folder = "../../_data/p-dengue/"
else:
    # server folder
    folder = "../../../data/lucaratzinger_data/p-dengue/"

valid_admin2 = pd.read_csv(os.path.join(folder, 'valid_admin/valid_admin2.csv'), header=None)[0].tolist()
valid_admin1 = pd.read_csv(os.path.join(folder, 'valid_admin/valid_admin1.csv'), header=None)[0].tolist()
valid_admin2.sort()
valid_admin1.sort()

In [59]:
#### shape data
admin1_shp = gpd.read_file(os.path.join(folder,"shapefiles/admin1/admin1_38.shp"))
admin2_shp = gpd.read_file(os.path.join(folder,"shapefiles/admin2/admin2_38.shp"))

admin1_shp = (admin1_shp[admin1_shp["admin1"].isin(valid_admin1)].sort_values("admin1").reset_index(drop=True))
admin2_shp = (admin2_shp[admin2_shp["admin2"].isin(valid_admin2)].sort_values("admin2").reset_index(drop=True))

print(f'shapefile crs: {admin1_shp.crs}')

#### Population raster
raster_pop = {year :  rxr.open_rasterio(os.path.join(folder, f'raster_pop/landscan-global-{year}.tif'), masked=True) for year in range(2015,2025)}
print(f'raster_pop crs: {raster_pop[2015].rio.crs}')

#### Urban raster (GHSL GHS-SMOD)
raster_urbanisation = rxr.open_rasterio(os.path.join(folder, 'raster_urbanisation/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V2_0.tif'), masked=True)

raster_urbanisation_binary = raster_urbanisation.copy()
urban_classes = [21, 22, 23, 30]
raster_urbanisation_binary = raster_urbanisation_binary.where(~raster_urbanisation_binary.isin(urban_classes), other=1)
rural_classes = [10, 11, 12, 13]
raster_urbanisation_binary = raster_urbanisation_binary.where(~raster_urbanisation_binary.isin(rural_classes), other=0)
raster_urbanisation_binary = raster_urbanisation_binary.where((raster_urbanisation_binary == 0) | (raster_urbanisation_binary == 1))
print(f'raster_urbanisation crs: {raster_urbanisation.rio.crs}')
raster_urbanisation = raster_urbanisation.rio.reproject(admin1_shp.crs)
raster_urbanisation_binary = raster_urbanisation_binary.rio.reproject(admin1_shp.crs)
print(f'raster_urbanisation crs projected to: {raster_urbanisation.rio.crs}')

#### Surveillance raster (Ahyoung Lim et al. Nature Medicine)
raster_surveillance = rxr.open_rasterio(os.path.join(folder, 'raster_surveillance/Surveillance_map_wmean.tif'), masked=True)
# EPSG:4326
raster_surveillance = raster_surveillance.rio.write_crs("EPSG:4326")
print(f'raster_surveillance crs: {raster_surveillance.rio.crs}')

shapefile crs: EPSG:4326
raster_pop crs: EPSG:4326
raster_urbanisation crs: ESRI:54009
raster_urbanisation crs projected to: EPSG:4326
raster_surveillance crs: EPSG:4326


In [60]:
# buffered region bounds
merged = admin1_shp.union_all()
merged_buffered = gpd.GeoDataFrame(geometry=[merged], crs=admin1_shp.crs)
merged_buffered["geometry"] = (merged_buffered.to_crs("EPSG:3857").buffer(20_000).to_crs("EPSG:4326"))
region_bounds_buffered = merged_buffered.bounds.iloc[0, :].to_list()

In [61]:
#### Population raster
raster_pop = {year :  raster_pop[year].squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3]) for year in range(2015,2025)}


#### Urban raster (GHSL GHS-SMOD)
raster_urbanisation_binary = raster_urbanisation_binary.squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])

#### Surveillance raster (Ahyoung Lim et al. Nature Medicine)
raster_surveillance = raster_surveillance.squeeze().rio.clip_box(minx=region_bounds_buffered[0], miny=region_bounds_buffered[1],
                                                   maxx=region_bounds_buffered[2], maxy=region_bounds_buffered[3])

---

### admin_year_pop

In [62]:
admin1_records = []
admin2_records = []

for year in range(2015,2025):
    admin1_pop_stats = zonal_stats(
        vectors=admin1_shp,
        raster=raster_pop[year].values,
        affine=raster_pop[year].rio.transform(),
        stats=["sum"],
        nodata=raster_pop[year].rio.nodata,
        all_touched=True)
    
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "population": admin1_pop_stats[i]["sum"]
        })

    admin2_pop_stats = zonal_stats(
        vectors=admin2_shp,
        raster=raster_pop[year].values,
        affine=raster_pop[year].rio.transform(),
        stats=["sum"],
        nodata=raster_pop[year].rio.nodata,
        all_touched=True)
    
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "population": admin2_pop_stats[i]["sum"]
        })

# Convert to DataFrames
df_admin1_pop = pd.DataFrame(admin1_records)
df_admin2_pop = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_pop = df_admin1_pop.reset_index(drop=True)
df_admin2_pop = df_admin2_pop.reset_index(drop=True)

In [63]:
df_admin2_pop

,admin2,year,population
0,ACEH BARAT,2015,287216.0
1,ACEH BARAT DAYA,2015,185923.0
2,ACEH BESAR,2015,464847.0
3,ACEH JAYA,2015,124867.0
4,ACEH SELATAN,2015,236026.0
...,...,...,...
5115,WAY KANAN,2024,496532.0
5116,WONOGIRI,2024,1181381.0
5117,WONOSOBO,2024,961265.0
5118,YAHUKIMO,2024,369804.0


In [64]:
df_admin2_pop.isna().sum()

admin2        0
year          0
population    0
dtype: int64

In [65]:
out_folder = os.path.join(folder, "admin_year_pop")
os.makedirs(out_folder, exist_ok=True)

# df_admin1_pop.to_csv(os.path.join(out_folder, "admin1_year_pop.csv"), index=False)
# df_admin2_pop.to_csv(os.path.join(out_folder, "admin2_year_pop.csv"), index=False)

---

### urbanisation

In [10]:
admin1_records = []
admin2_records = []

# population-aligned urbanisation raster
_, raster_urbanisation_binary_pop_aligned = best_res_align(raster_pop[2015], '',
                                                    raster_urbanisation_binary, 'nearest',
                                                    region_bounds_buffered = region_bounds_buffered,
                                                    shape_crs = admin1_shp.crs, crop=False)

In [11]:
for year in range(2015,2025):
    # admin1
    # Non-weighted: simple mean over each polygon
    urbanisation_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urbanisation_binary_pop_aligned.values,
        affine=raster_urbanisation_binary_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urbanisation_binary_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(urban * pop) / sum(pop)
    urbanisation_pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urbanisation_binary_pop_aligned.shape[-2:],
                         transform=raster_urbanisation_binary_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urbanisation_values = raster_urbanisation_binary_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urbanisation_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = urbanisation_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(urbanisation_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urbanisation_pop_weighted_records.append(weighted_mean)

    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "urbanisation_nonweighted": urbanisation_nonweighted[i]["mean"],
            "urbanisation_pop_weighted": urbanisation_pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    urbanisation_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urbanisation_binary_pop_aligned.values,
        affine=raster_urbanisation_binary_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urbanisation_binary_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(urban * pop) / sum(pop)
    urbanisation_pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urbanisation_binary_pop_aligned.shape[-2:],
                         transform=raster_urbanisation_binary_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urbanisation_values = raster_urbanisation_binary_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urbanisation_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = urbanisation_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(urbanisation_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urbanisation_pop_weighted_records.append(weighted_mean)

    # Store results in admin2_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "urbanisation_nonweighted": urbanisation_nonweighted[i]["mean"],
            "urbanisation_pop_weighted": urbanisation_pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_urbanisation = pd.DataFrame(admin1_records)
df_admin2_urbanisation = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_urbanisation = df_admin1_urbanisation.reset_index(drop=True)
df_admin2_urbanisation = df_admin2_urbanisation.reset_index(drop=True)

In [12]:
df_admin2_urbanisation

,admin2,year,urbanisation_nonweighted,urbanisation_pop_weighted
0,ACEH BARAT,2015,0.018364,0.224615
1,ACEH BARAT DAYA,2015,0.037194,0.666287
2,ACEH BESAR,2015,0.099596,0.589342
3,ACEH JAYA,2015,0.009563,0.256873
4,ACEH SELATAN,2015,0.038308,0.405260
...,...,...,...,...
5115,WAY KANAN,2024,0.045294,0.231300
5116,WONOGIRI,2024,0.453802,0.694331
5117,WONOSOBO,2024,0.513472,0.795368
5118,YAHUKIMO,2024,0.008076,0.069001


In [13]:
df_admin2_urbanisation.isna().sum()

admin2                       0
year                         0
urbanisation_nonweighted     0
urbanisation_pop_weighted    0
dtype: int64

In [15]:
non_urban_admin2 = df_admin2_urbanisation.loc[(df_admin2_urbanisation['urbanisation_nonweighted']==0)&(df_admin2_urbanisation['year']==2015), 'admin2'].to_list()
non_urban_admin2

['KONAWE KEPULAUAN',
 'KONAWE UTARA',
 'MAHAKAM HULU',
 'MANOKWARI SELATAN',
 'MAYBRAT',
 'PAKPAK BHARAT',
 'PEGUNUNGAN ARFAK',
 'TAMBRAUW',
 'TANA TIDUNG']

In [ ]:
out_folder = os.path.join(folder, "admin_year_urbanisation")
os.makedirs(out_folder, exist_ok=True)

# df_admin1_urbanisation.to_csv(os.path.join(out_folder, "admin1_year_urbanisation.csv"), index=False)
# df_admin2_urbanisation.to_csv(os.path.join(out_folder, "admin2_year_urbanisation.csv"), index=False)

---

### surveillance

In [17]:
admin1_records = []
admin2_records = []

# population-aligned surveillance raster
_, raster_surveillance_pop_aligned = best_res_align(raster_pop[2015], '',
                                                    raster_surveillance, 'nearest',
                                                    region_bounds_buffered = region_bounds_buffered,
                                                    shape_crs = admin1_shp.crs)

In [18]:
for year in range(2015,2025):
    # admin1
    # Non-weighted: simple mean over each polygon
    surveillance_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_surveillance_pop_aligned.values,
        affine=raster_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_surveillance_pop_aligned.shape[-2:],
                         transform=raster_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        surveillance_values = raster_surveillance_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(surveillance_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = surveillance_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        pop_weighted_records.append(weighted_mean)

    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "surveillance_nonweighted": surveillance_nonweighted[i]["mean"],
            "surveillance_pop_weighted": pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    surveillance_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_surveillance_pop_aligned.values,
        affine=raster_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_surveillance_pop_aligned.shape[-2:],
                         transform=raster_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        surveillance_values = raster_surveillance_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(surveillance_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = surveillance_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        pop_weighted_records.append(weighted_mean)
        
    # Store results in admin1_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "surveillance_nonweighted": surveillance_nonweighted[i]["mean"],
            "surveillance_pop_weighted": pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_surveillance = pd.DataFrame(admin1_records)
df_admin2_surveillance = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_surveillance = df_admin1_surveillance.reset_index(drop=True)
df_admin2_surveillance = df_admin2_surveillance.reset_index(drop=True)

In [19]:
df_admin2_surveillance

,admin2,year,surveillance_nonweighted,surveillance_pop_weighted
0,ACEH BARAT,2015,0.369001,0.639564
1,ACEH BARAT DAYA,2015,0.297746,0.740813
2,ACEH BESAR,2015,0.510617,0.858704
3,ACEH JAYA,2015,0.236977,0.572931
4,ACEH SELATAN,2015,0.233088,0.498598
...,...,...,...,...
5115,WAY KANAN,2024,0.515941,0.606863
5116,WONOGIRI,2024,0.838876,0.870500
5117,WONOSOBO,2024,0.848702,0.882246
5118,YAHUKIMO,2024,0.057116,0.077861


In [20]:
df_admin2_surveillance.isna().sum()

admin2                       0
year                         0
surveillance_nonweighted     0
surveillance_pop_weighted    0
dtype: int64

In [ ]:
out_folder = os.path.join(folder, "admin_year_surveillance")
os.makedirs(out_folder, exist_ok=True)

# df_admin1_surveillance.to_csv(os.path.join(out_folder, "admin1_year_surveillance.csv"), index=False)
# df_admin2_surveillance.to_csv(os.path.join(out_folder, "admin2_year_surveillance.csv"), index=False)

---

### urban surveillance

In [22]:
admin1_records = []
admin2_records = []
raster_urbanisation_binary_pop_aligned_mask = raster_urbanisation_binary_pop_aligned.where(raster_urbanisation_binary_pop_aligned==1, other=np.nan)
raster_urban_surveillance_pop_aligned = raster_surveillance_pop_aligned * raster_urbanisation_binary_pop_aligned_mask
raster_urban_surveillance_pop_aligned = raster_urban_surveillance_pop_aligned.rio.write_nodata(np.nan, inplace=False)

In [23]:
for year in range(2015,2025):
    # admin1
    # Non-weighted: simple mean over each polygon
    urban_surveillance_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urban_surveillance_pop_aligned.values,
        affine=raster_urban_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urban_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    urban_surveillance_pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urban_surveillance_pop_aligned.shape[-2:],
                         transform=raster_urban_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urban_surveillance_values = raster_urban_surveillance_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urban_surveillance_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = urban_surveillance_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(urban_surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urban_surveillance_pop_weighted_records.append(weighted_mean)
        
    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "urban_surveillance_nonweighted": urban_surveillance_nonweighted[i]["mean"],
            "urban_surveillance_pop_weighted": urban_surveillance_pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    urban_surveillance_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urban_surveillance_pop_aligned.values,
        affine=raster_urban_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urban_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    urban_surveillance_pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urban_surveillance_pop_aligned.shape[-2:],
                         transform=raster_urban_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urban_surveillance_values = raster_urban_surveillance_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urban_surveillance_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = urban_surveillance_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(urban_surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urban_surveillance_pop_weighted_records.append(weighted_mean)
        
    # Store results in admin2_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "urban_surveillance_nonweighted": urban_surveillance_nonweighted[i]["mean"],
            "urban_surveillance_pop_weighted": urban_surveillance_pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_urban_surveillance = pd.DataFrame(admin1_records)
df_admin2_urban_surveillance = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_urban_surveillance = df_admin1_urban_surveillance.reset_index(drop=True)
df_admin2_urban_surveillance = df_admin2_urban_surveillance.reset_index(drop=True)

In [56]:
admin1_records = []
admin2_records = []
raster_urbanisation_binary_pop_aligned_mask = raster_urbanisation_binary_pop_aligned.where(raster_urbanisation_binary_pop_aligned==1, other=np.nan)
raster_urban_surveillance_pop_aligned = raster_surveillance_pop_aligned * raster_urbanisation_binary_pop_aligned_mask
raster_urban_surveillance_pop_aligned = raster_urban_surveillance_pop_aligned.rio.write_nodata(np.nan, inplace=False)

for year in range(2015,2025):
    # admin1
    # Non-weighted: simple mean over each polygon
    urban_surveillance_nonweighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urban_surveillance_pop_aligned.values,
        affine=raster_urban_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urban_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    urban_surveillance_pop_weighted_records = []
    for i, row in enumerate(admin1_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urban_surveillance_pop_aligned.shape[-2:],
                         transform=raster_urban_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urban_surveillance_values = raster_urban_surveillance_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urban_surveillance_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = urban_surveillance_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(urban_surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urban_surveillance_pop_weighted_records.append(weighted_mean)
        
    # Store results in admin1_records
    for i, row in enumerate(admin1_shp.itertuples()):
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "urban_surveillance_nonweighted": urban_surveillance_nonweighted[i]["mean"],
            "urban_surveillance_pop_weighted": urban_surveillance_pop_weighted_records[i]
        })

    # admin2
    # Non-weighted: simple mean over each polygon
    urban_surveillance_nonweighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urban_surveillance_pop_aligned.values,
        affine=raster_urban_surveillance_pop_aligned.rio.transform(),
        stats=["mean"],
        nodata=raster_urban_surveillance_pop_aligned.rio.nodata,
        all_touched=True
    )

    # Population-weighted: sum(surveillance * pop) / sum(pop)
    urban_surveillance_pop_weighted_records = []
    for i, row in enumerate(admin2_shp.itertuples()):
        mask = rasterize([(row.geometry, 1)],
                         out_shape=raster_urban_surveillance_pop_aligned.shape[-2:],
                         transform=raster_urban_surveillance_pop_aligned.rio.transform(),
                         fill=0,
                         all_touched=True,
                         dtype="uint8"
                         ).astype(bool)
        urban_surveillance_values = raster_urban_surveillance_pop_aligned.values[mask]
        pop_values = raster_pop[year].values[mask]

        # Keep only cells where BOTH are valid
        valid = (~np.isnan(urban_surveillance_values)) & (~np.isnan(pop_values))

        if not np.any(valid):
            weighted_mean = np.nan
        else:
            if np.sum(pop_values[valid]) == 0:
                weighted_mean = urban_surveillance_nonweighted[i]["mean"]
            else:
                weighted_mean = np.sum(urban_surveillance_values[valid] * pop_values[valid]) / np.sum(pop_values[valid])
        urban_surveillance_pop_weighted_records.append(weighted_mean)
        
    # Store results in admin2_records
    for i, row in enumerate(admin2_shp.itertuples()):
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "urban_surveillance_nonweighted": urban_surveillance_nonweighted[i]["mean"],
            "urban_surveillance_pop_weighted": urban_surveillance_pop_weighted_records[i]
        })

# Convert to DataFrames
df_admin1_urban_surveillance = pd.DataFrame(admin1_records)
df_admin2_urban_surveillance = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_urban_surveillance = df_admin1_urban_surveillance.reset_index(drop=True)
df_admin2_urban_surveillance = df_admin2_urban_surveillance.reset_index(drop=True)

KeyboardInterrupt: 

In [25]:
for a2 in non_urban_admin2:
    df_admin2_urban_surveillance.loc[df_admin2_urban_surveillance['admin2']==a2, 'urban_surveillance_nonweighted'] = df_admin2_surveillance[df_admin2_surveillance['admin2']==a2]['surveillance_nonweighted']
    df_admin2_urban_surveillance.loc[df_admin2_urban_surveillance['admin2']==a2, 'urban_surveillance_pop_weighted'] = df_admin2_surveillance[df_admin2_surveillance['admin2']==a2]['surveillance_pop_weighted']

In [26]:
no_urban_surveillance_coincidence_admin2 = df_admin2_urban_surveillance.loc[(df_admin2_urban_surveillance['urban_surveillance_nonweighted'].isna()) & (df_admin2_urban_surveillance['year']==2015), 'admin2'].to_list()

In [27]:
for a2 in no_urban_surveillance_coincidence_admin2:
    df_admin2_urban_surveillance.loc[df_admin2_urban_surveillance['admin2']==a2, 'urban_surveillance_nonweighted'] = df_admin2_surveillance[df_admin2_surveillance['admin2']==a2]['surveillance_nonweighted']
    df_admin2_urban_surveillance.loc[df_admin2_urban_surveillance['admin2']==a2, 'urban_surveillance_pop_weighted'] = df_admin2_surveillance[df_admin2_surveillance['admin2']==a2]['surveillance_pop_weighted']

In [28]:
df_admin2_urban_surveillance

,admin2,year,urban_surveillance_nonweighted,urban_surveillance_pop_weighted
0,ACEH BARAT,2015,0.829057,0.787234
1,ACEH BARAT DAYA,2015,0.825269,0.844500
2,ACEH BESAR,2015,0.915559,0.939699
3,ACEH JAYA,2015,0.864509,0.879474
4,ACEH SELATAN,2015,0.694324,0.680381
...,...,...,...,...
5115,WAY KANAN,2024,0.765571,0.760155
5116,WONOGIRI,2024,0.868872,0.888670
5117,WONOSOBO,2024,0.864985,0.891884
5118,YAHUKIMO,2024,0.171283,0.184696


In [29]:
df_admin2_urban_surveillance.isna().sum()

admin2                             0
year                               0
urban_surveillance_nonweighted     0
urban_surveillance_pop_weighted    0
dtype: int64

In [ ]:
out_folder = os.path.join(folder, "admin_year_urban_surveillance")
os.makedirs(out_folder, exist_ok=True)

# df_admin1_urban_surveillance.to_csv(os.path.join(out_folder, "admin1_year_urban_surveillance.csv"), index=False)
# df_admin2_urban_surveillance.to_csv(os.path.join(out_folder, "admin2_year_urban_surveillance.csv"), index=False)

---

### density and urban density

In [68]:
admin1_records = []
admin2_records = []

for year in range(2015,2025):
    _, raster_urbanisation_binary_pop_aligned = best_res_align(raster_pop[year], '',
                                                    raster_urbanisation_binary, 'nearest',
                                                    region_bounds_buffered = region_bounds_buffered,
                                                    shape_crs = admin1_shp.crs, crop=False)
    raster_urbanisation_binary_pop_aligned_mask = raster_urbanisation_binary_pop_aligned.where(raster_urbanisation_binary_pop_aligned==1, other=np.nan)
    raster_urban_pop_pop_aligned = raster_pop[year] * raster_urbanisation_binary_pop_aligned_mask
    raster_urban_pop_pop_aligned = raster_urban_pop_pop_aligned.rio.write_nodata(np.nan, inplace=False)
    ###
    admin1_pop_stats = zonal_stats(
        vectors=admin1_shp,
        raster=raster_pop[year].values,
        affine=raster_pop[year].rio.transform(),
        stats=["mean", "sum"],
        nodata=raster_pop[year].rio.nodata,
        all_touched=True)
    
    admin1_pop_stats_pop_weighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_pop[year].values**2,
        affine=raster_pop[year].rio.transform(),
        stats=["sum"],
        nodata=raster_pop[year].rio.nodata,
        all_touched=True)
    
    admin1_urban_pop_stats = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urban_pop_pop_aligned.values,
        affine=raster_urban_pop_pop_aligned.rio.transform(),
        stats=["mean", "sum"],
        nodata=raster_urban_pop_pop_aligned.rio.nodata,
        all_touched=True)
    
    admin1_urban_pop_stats_pop_weighted = zonal_stats(
        vectors=admin1_shp,
        raster=raster_urban_pop_pop_aligned.values*raster_pop[year].values,
        affine=raster_urban_pop_pop_aligned.rio.transform(),
        stats=["sum"],
        nodata=raster_urban_pop_pop_aligned.rio.nodata,
        all_touched=True)

    for i, row in enumerate(admin1_shp.itertuples()):
        density_unweighted = admin1_pop_stats[i]["mean"]
        density_pop_weighted = admin1_pop_stats_pop_weighted[i]["sum"]/admin1_pop_stats[i]["sum"]
        urban_pop_sum = admin1_urban_pop_stats[i]["sum"]
        urban_density_unweighted = (admin1_urban_pop_stats[i]["mean"]
                                      if urban_pop_sum and urban_pop_sum > 0
                                      else density_unweighted)
        urban_density_pop_weighted = (admin1_urban_pop_stats_pop_weighted[i]["sum"] / urban_pop_sum
                                      if urban_pop_sum and urban_pop_sum > 0
                                      else density_pop_weighted)
        
        admin1_records.append({
            "admin1": row.admin1,
            "year": year,
            "density_unweighted": density_unweighted,
            "density_pop_weighted": density_pop_weighted,
            "urban_density_unweighted": urban_density_unweighted,
            "urban_density_pop_weighted": urban_density_pop_weighted
        })
    ###
    admin2_pop_stats = zonal_stats(
        vectors=admin2_shp,
        raster=raster_pop[year].values,
        affine=raster_pop[year].rio.transform(),
        stats=["mean", "sum"],
        nodata=raster_pop[year].rio.nodata,
        all_touched=True)
    
    admin2_pop_stats_pop_weighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_pop[year].values**2,
        affine=raster_pop[year].rio.transform(),
        stats=["sum"],
        nodata=raster_pop[year].rio.nodata,
        all_touched=True)
    
    admin2_urban_pop_stats = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urban_pop_pop_aligned.values,
        affine=raster_urban_pop_pop_aligned.rio.transform(),
        stats=["mean", "sum"],
        nodata=raster_urban_pop_pop_aligned.rio.nodata,
        all_touched=True)
    
    admin2_urban_pop_stats_pop_weighted = zonal_stats(
        vectors=admin2_shp,
        raster=raster_urban_pop_pop_aligned.values*raster_pop[year].values,
        affine=raster_urban_pop_pop_aligned.rio.transform(),
        stats=["sum"],
        nodata=raster_urban_pop_pop_aligned.rio.nodata,
        all_touched=True)

    for i, row in enumerate(admin2_shp.itertuples()):
        density_unweighted = admin2_pop_stats[i]["mean"]
        density_pop_weighted = admin2_pop_stats_pop_weighted[i]["sum"]/admin2_pop_stats[i]["sum"]
        urban_pop_sum = admin2_urban_pop_stats[i]["sum"]
        urban_density_unweighted = (admin2_urban_pop_stats[i]["mean"]
                                      if urban_pop_sum and urban_pop_sum > 0
                                      else density_unweighted)
        urban_density_pop_weighted = (admin2_urban_pop_stats_pop_weighted[i]["sum"] / urban_pop_sum
                                      if urban_pop_sum and urban_pop_sum > 0
                                      else density_pop_weighted)
        
        admin2_records.append({
            "admin2": row.admin2,
            "year": year,
            "density_unweighted": density_unweighted,
            "density_pop_weighted": density_pop_weighted,
            "urban_density_unweighted": urban_density_unweighted,
            "urban_density_pop_weighted": urban_density_pop_weighted
        })

# Convert to DataFrames
df_admin1_density = pd.DataFrame(admin1_records)
df_admin2_density = pd.DataFrame(admin2_records)

# Optional: reset index
df_admin1_density = df_admin1_density.reset_index(drop=True)
df_admin2_density = df_admin2_density.reset_index(drop=True)

In [70]:
df_admin1_density

,admin1,year,density_unweighted,density_pop_weighted,urban_density_unweighted,urban_density_pop_weighted
0,ACEH,2015,71.686807,4033.354864,785.036964,5844.068688
1,BALI,2015,618.357923,8649.629506,1172.929178,10401.391277
2,BANTEN,2015,1057.990321,8283.968398,1917.369398,9273.038596
3,BENGKULU,2015,78.584324,7648.336261,907.863102,16304.408038
4,DAERAH ISTIMEWA YOGYAKARTA,2015,980.960577,5261.463427,1457.707535,5991.956726
...,...,...,...,...,...,...
375,SULAWESI TENGGARA,2024,61.321838,1377.780961,823.899154,2486.427126
376,SULAWESI UTARA,2024,151.770204,2975.487418,1362.091060,4452.044728
377,SUMATRA BARAT,2024,114.423539,2566.633460,980.088864,4018.633226
378,SUMATRA SELATAN,2024,86.169976,3762.929758,1285.764857,7778.548644


In [ ]:
out_folder = os.path.join(folder, "admin_year_density")
os.makedirs(out_folder, exist_ok=True)

df_admin1_density.to_csv(os.path.join(out_folder, "admin1_year_density.csv"), index=False)
df_admin2_density.to_csv(os.path.join(out_folder, "admin2_year_density.csv"), index=False)

---

### climate variables

In [31]:
weather_folder = os.path.join(folder, "weather_statistics-(reanalysis-era5-land)")

In [32]:
t2m_monthly_mean = {}
t2m_monthly_min = {}
t2m_monthly_max = {}
tp_monthly_mean = {}
rh_monthly_mean = {}

pixel_monthly_statistics = {}

t2m_monthly_mean_aligned = {}
t2m_monthly_min_aligned = {}
t2m_monthly_max_aligned = {}
tp_monthly_mean_aligned = {}
rh_monthly_mean_aligned = {}

pixel_monthly_statistics_aligned = {}

stat_names = ["t2m_mean", "t2m_min", "t2m_max", "tp_mean", "rh_mean"]

In [35]:
for year in range(2015,2025):
    # read in climate data datasets
    t2m = xr.open_dataset(os.path.join(weather_folder, f"t2m/t2m_{year}.grib"),engine="cfgrib")
    t2m = t2m.isel(time=slice(1, None))
    t2m = t2m['t2m']
    t2m = t2m.rio.write_crs("EPSG:4326")
    tp = xr.open_dataset(os.path.join(weather_folder, f"tp/tp_{year}.grib"),engine="cfgrib")
    tp = tp.isel(time=slice(1, None))
    tp = tp['tp']
    tp = tp.rio.write_crs("EPSG:4326")
    rh = xr.open_dataset(os.path.join(weather_folder, f"rh/rh_{year}.nc"),engine="netcdf4")
    rh = rh['rh']
    rh = rh.rio.write_crs("EPSG:4326")

    # calculate monthly pixel statistics
    t2m['time'] = pd.to_datetime(t2m['time'].values)
    t2m_monthly_mean[year] = t2m.groupby('time.month').mean(dim=['time', 'step'])
    t2m_monthly_min[year] = t2m.groupby('time.month').min(dim=['time', 'step'])
    t2m_monthly_max[year] = t2m.groupby('time.month').max(dim=['time', 'step'])

    tp['time'] = pd.to_datetime(tp['time'].values)
    tp_monthly_mean[year] = tp.groupby('time.month').mean(dim=['time', 'step'])

    rh['time'] = pd.to_datetime(rh['time'].values)
    rh_monthly_mean[year] = rh.groupby('time.month').mean(dim=['time', 'step'])

    # all nearest because non land values are nan and will bleed inside when interpolating
    align_methods = {"t2m_mean": "nearest",
                     "t2m_min":  "nearest",
                     "t2m_max":  "nearest",
                     "tp_mean":  "nearest",
                     "rh_mean":  "nearest"}
    
    _, t2m_monthly_mean_aligned[year] = best_res_align(raster_pop[year], '',
                                            t2m_monthly_mean[year], align_methods['t2m_mean'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, t2m_monthly_min_aligned[year] = best_res_align(raster_pop[year], '',
                                            t2m_monthly_min[year], align_methods['t2m_min'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, t2m_monthly_max_aligned[year] = best_res_align(raster_pop[year], '',
                                            t2m_monthly_max[year], align_methods['t2m_max'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, tp_monthly_mean_aligned[year] = best_res_align(raster_pop[year], '',
                                            tp_monthly_mean[year], align_methods['tp_mean'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)
    _, rh_monthly_mean_aligned[year] = best_res_align(raster_pop[year], '',
                                            rh_monthly_mean[year], align_methods['rh_mean'],
                                            region_bounds_buffered = region_bounds_buffered,
                                            shape_crs = admin1_shp.crs)

pixel_monthly_statistics = {"t2m_mean" : t2m_monthly_mean,
                            "t2m_min" : t2m_monthly_min,
                            "t2m_max" : t2m_monthly_max,
                            "tp_mean" : tp_monthly_mean,
                            "rh_mean" : rh_monthly_mean}
pixel_monthly_statistics_aligned = {"t2m_mean" : t2m_monthly_mean_aligned,
                                    "t2m_min" : t2m_monthly_min_aligned,
                                    "t2m_max" : t2m_monthly_max_aligned,
                                    "tp_mean" : tp_monthly_mean_aligned,
                                    "rh_mean" : rh_monthly_mean_aligned}

### Masking and averaging

In [51]:
admin1_dict = {}
admin2_dict = {}
for stat_name in stat_names[:]:
    print(stat_name)
    for year in range(2015,2025):
        print(year)
        pixel_statistic_aligned = pixel_monthly_statistics_aligned[stat_name][year]

        # admin1
        statistic_unweighted_records_1 = []
        statistic_pop_weighted_records_1 = []

        for i, row in enumerate(admin1_shp.itertuples()):
            print(row.admin1)
            mask = rasterize([(row.geometry, 1)],
                                out_shape=pixel_statistic_aligned.shape[-2:],
                                transform=pixel_statistic_aligned.rio.transform(),
                                fill=0,
                                all_touched=True,
                                dtype="uint8"
                                ).astype(bool)
            statistic_values = np.where(mask, pixel_statistic_aligned.values, np.nan)
            pop_values = np.where(mask, raster_pop[year].values, np.nan)
            pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
            if np.all(np.isnan(statistic_values)):
                print('all nan')
                continue
            unweighted_mean_per_month = np.nanmean(statistic_values, axis=(1,2))
            statistic_unweighted_records_1.append(unweighted_mean_per_month)

            valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
            weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(1,2))
            pop_sum = np.nansum(pop_values_b * valid, axis=(1,2))
            
            pop_weighted_mean_per_month = weighted_sum / pop_sum
            statistic_pop_weighted_records_1.append(pop_weighted_mean_per_month)

            for month in range(1, statistic_values.shape[0]+1):
                key = (row.admin1, year, month)
                if key not in admin1_dict:
                    admin1_dict[key] = {
                        "admin1": row.admin1,
                        "year": year,
                        "month": month
                    }
                admin1_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[month-1]
                admin1_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[month-1]

        # admin2
        statistic_unweighted_records_2 = []
        statistic_pop_weighted_records_2 = []

        for i, row in enumerate(admin2_shp.itertuples()):
            print(row.admin2)
            mask = rasterize([(row.geometry, 1)],
                                out_shape=pixel_statistic_aligned.shape[-2:],
                                transform=pixel_statistic_aligned.rio.transform(),
                                fill=0,
                                all_touched=True,
                                dtype="uint8"
                                ).astype(bool)
            statistic_values = np.where(mask, pixel_statistic_aligned.values, np.nan)
            pop_values = np.where(mask, raster_pop[year].values, np.nan)
            pop_values_b = np.broadcast_to(pop_values, statistic_values.shape)
            if np.all(np.isnan(statistic_values)):
                print('all nan')
                continue
            unweighted_mean_per_month = np.nanmean(statistic_values, axis=(1,2))
            statistic_unweighted_records_2.append(unweighted_mean_per_month)

            valid = (~np.isnan(statistic_values)) & (~np.isnan(pop_values_b))
            weighted_sum = np.nansum(statistic_values * pop_values_b * valid, axis=(1,2))
            pop_sum = np.nansum(pop_values_b * valid, axis=(1,2))
            
            pop_weighted_mean_per_month = weighted_sum / pop_sum
            statistic_pop_weighted_records_2.append(pop_weighted_mean_per_month)

            for month in range(1, statistic_values.shape[0]+1):
                key = (row.admin2, year, month)
                if key not in admin2_dict:
                    admin2_dict[key] = {
                        "admin2": row.admin2,
                        "year": year,
                        "month": month
                    }
                admin2_dict[key][f"{stat_name}_unweighted"] = unweighted_mean_per_month[month-1]
                admin2_dict[key][f"{stat_name}_pop_weighted"] = pop_weighted_mean_per_month[month-1]

t2m_mean
2015
ACEH
BALI
BANTEN
BENGKULU
DAERAH ISTIMEWA YOGYAKARTA
DKI JAKARTA
GORONTALO
JAMBI
JAWA BARAT
JAWA TENGAH
JAWA TIMUR
KALIMANTAN BARAT
KALIMANTAN SELATAN
KALIMANTAN TENGAH
KALIMANTAN TIMUR
KALIMANTAN UTARA
KEPULAUAN BANGKA BELITUNG
KEPULAUAN RIAU
LAMPUNG
MALUKU
MALUKU UTARA
NUSA TENGGARA BARAT
NUSA TENGGARA TIMUR
PAPUA
PAPUA BARAT
PAPUA BARAT DAYA
PAPUA PEGUNUNGAN
PAPUA SELATAN
PAPUA TENGAH
RIAU
SULAWESI BARAT
SULAWESI SELATAN
SULAWESI TENGAH
SULAWESI TENGGARA
SULAWESI UTARA
SUMATRA BARAT
SUMATRA SELATAN
SUMATRA UTARA
ACEH BARAT
ACEH BARAT DAYA
ACEH BESAR
ACEH JAYA
ACEH SELATAN
ACEH SINGKIL
ACEH TAMIANG
ACEH TENGAH
ACEH TENGGARA
ACEH TIMUR
ACEH UTARA
AGAM
ALOR
ASAHAN
ASMAT
BADUNG
BALANGAN
BANDUNG
BANDUNG BARAT
BANGGAI
BANGGAI KEPULAUAN
BANGGAI LAUT
BANGKA
BANGKA BARAT
BANGKA SELATAN
BANGKA TENGAH
BANGKALAN
BANGLI
BANJAR
BANJARNEGARA
BANTAENG
BANTUL
BANYU ASIN
BANYUMAS
BANYUWANGI
BARITO KUALA
BARITO SELATAN
BARITO TIMUR
BARITO UTARA
BARRU
BATANG
BATANG HARI
BATU BARA
BEKASI
B

In [52]:
# Convert to DataFrame
admin1_records = list(admin1_dict.values())
df_admin1_statistics = pd.DataFrame(admin1_records)
df_admin1_statistics = df_admin1_statistics.sort_values(by=["admin1", "year", "month"]).reset_index(drop=True)

admin2_records = list(admin2_dict.values())
df_admin2_statistics = pd.DataFrame(admin2_records)
df_admin2_statistics = df_admin2_statistics.sort_values(by=["admin2", "year", "month"]).reset_index(drop=True)

In [53]:
df_admin2_statistics

,admin2,year,month,t2m_mean_unweighted,t2m_mean_pop_weighted,t2m_min_unweighted,t2m_min_pop_weighted,t2m_max_unweighted,t2m_max_pop_weighted,tp_mean_unweighted,tp_mean_pop_weighted,rh_mean_unweighted,rh_mean_pop_weighted
0,ACEH BARAT,2015,1,296.553192,298.113488,292.452057,294.097140,302.009125,302.825969,0.005766,0.004416,89.093109,87.167199
1,ACEH BARAT,2015,2,296.917328,298.429072,291.295868,293.372377,302.661713,303.363930,0.005092,0.004271,86.445282,85.063198
2,ACEH BARAT,2015,3,297.053772,298.567195,292.830322,294.606322,302.866943,303.605977,0.004616,0.003547,88.586975,87.120348
3,ACEH BARAT,2015,4,296.811218,298.293407,293.230194,294.852081,302.896729,303.613153,0.008693,0.007106,91.291878,89.598911
4,ACEH BARAT,2015,5,297.676849,299.290881,293.623840,295.463444,303.104401,303.967454,0.003813,0.002801,88.033737,85.696828
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61435,YALIMO,2024,8,294.672638,294.702880,290.136902,289.959379,299.825897,299.376732,0.004374,0.003554,86.708893,85.686131
61436,YALIMO,2024,9,294.606079,294.658601,289.825562,289.764754,300.661285,300.381228,0.006410,0.005784,87.778755,86.769433
61437,YALIMO,2024,10,294.584778,294.678387,290.242371,290.284942,300.648193,300.368047,0.008298,0.007836,87.763924,86.740003
61438,YALIMO,2024,11,294.789398,294.851689,289.383331,289.130681,301.835602,301.330760,0.006114,0.005306,87.147903,86.362742


In [ ]:
# df_admin1_statistics.to_csv(os.path.join(folder, 'admin_year_month_climate_statistics/admin1_year_month_climate_statistics.csv'), index=False)
# df_admin2_statistics.to_csv(os.path.join(folder, 'admin_year_month_climate_statistics/admin2_year_month_climate_statistics.csv'), index=False)

---